# В этом ноутбуке эксперименты, связанные с инъекцией вектора eGeMAPS в полносвязный слой нейронной сети. <br>
### Гипотеза: это поможет улучшить результат по сравнению с только лишь спектрограммами.

In [1]:
from models.models_one_task_egemaps import AlexNetEgemaps2048, AlexNetEgemaps1792
from  datasets.iemocap import IemocapDataset, train_test_loaders
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
from skorch.classifier import NeuralNetClassifier
import skorch.callbacks as callbacks
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from IPython.display import display
import pandas as pd

## Модель 
Используется AlexNet, как модель, показавшая один из наилучших результатов. Позже также проведу этот эксперимент на VGG-11. Отобразим результаты этих моделей.

In [2]:
df = pd.read_csv(os.path.join(RESULTS_FOLDER, 'iemocap_all_results.csv'), delimiter=';')
alex_results = df.loc[(df['model'] == 'AlexNet') & (df['preprocessing'] == False) & (df['augmentation'] == True) ]
display(alex_results.loc[alex_results['epochs'] == 172])
vgg_results = df.loc[(df['model'] == 'VggNet') & (df['preprocessing'] == False) & (df['augmentation'] == True)]
display(vgg_results.loc[vgg_results['epochs'] == 210])

,result,epochs,subset,metric,model,dataset,preprocessing,augmentation
12731,0.701568,172,train,acc,AlexNet,IEMOCAP-4,False,True
12909,0.683091,172,valid,acc,AlexNet,IEMOCAP-4,False,True
13087,0.741412,172,train,loss,AlexNet,IEMOCAP-4,False,True
13265,0.808721,172,valid,loss,AlexNet,IEMOCAP-4,False,True


,result,epochs,subset,metric,model,dataset,preprocessing,augmentation
6037,0.739362,210,train,acc,VggNet,IEMOCAP-4,False,True
6283,0.683091,210,valid,acc,VggNet,IEMOCAP-4,False,True
6529,0.671230,210,train,loss,VggNet,IEMOCAP-4,False,True
6775,0.794945,210,valid,loss,VggNet,IEMOCAP-4,False,True


## Вопрос: куда мы будем конкатенировать eGeMAPS? 
У модели AlexNet три полносвязных слоя: 6400 нейронов, 2048 нейронов, 512 нейронов, 4 нейрона (классы) <br>
Для начала я попробую конкатенировать слой с 2048 нейронов. Есть и другие варианты, но начнем с простого.<br>
Этот слой будет подвергнут нормализации (torch.nn.LayerNorm). 

### 1. Конкатенация со слоем 2048 нейронов, LayerNorm

In [3]:
model = AlexNetEgemaps2048(num_classes=4)
print(model)

AlexNetEgemaps2048(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
  (fc1): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=6400, out_features=2048, bias=Tru

## Датасет
Используется IEMOCAP без препроцессинга и без аугментацией данных.

In [2]:
train_ds = IemocapDataset( 
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks='emotion', egemaps=True 
)
valid_ds = IemocapDataset(  
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=False, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks='emotion', egemaps=True 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


## Обучение
Будем обучать с помощью Skorch, используем оптимизатор Adam, learning rate 1e-5, 300 эпох (сохраняем все гиперпараметры такими же, как и у бейслайна)

In [5]:
filename = 'AlexNetEgemaps2048LN--{}_augmentation-{}.md'.format(train_ds.name, str(train_ds.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [6]:
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(valid_ds), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [7]:
net.fit(train_ds, None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.3519        1.3542       0.4211        1.2350     +  18.6267
      2       0.4516        1.2440       0.5185        1.1522     +  18.3658
      3       0.5003        1.1453       0.5745        1.0373     +  18.2437
      4       0.5428        1.0839       0.5823        1.0026     +  18.2562
      5       0.5622        1.0618       0.5633        1.0266        18.3714
      6       0.5608        1.0459       0.5857        0.9779     +  18.3271
      7       0.5719        1.0378       0.5845        0.9885        18.5022
      8       0.5767        1.0237       0.6114        0.9656     +  18.4624
      9       0.5885        1.0165       0.6025        0.9581     +  18.4088
     10       0.5938        1.0079       0.6036        0.9583        18.5729
     11       0.5851        1.0004       0.6013        0.9516     +  18.4611

     88       0.7427        0.6662       0.6842        0.8163        18.4531
     89       0.7492        0.6466       0.6898        0.8071        18.5458
     90       0.7402        0.6645       0.6719        0.8440        18.4693
     91       0.7531        0.6443       0.6741        0.8162        18.4278
     92       0.7623        0.6320       0.6876        0.8144        18.3892
     93       0.7595        0.6264       0.6775        0.8075        18.3617
     94       0.7612        0.6188       0.6898        0.8254        18.3706
     95       0.7618        0.6184       0.6920        0.8247        18.4175
     96       0.7595        0.6153       0.6506        0.9086        18.2834
     97       0.7592        0.6183       0.6909        0.8360        18.3771
     98       0.7696        0.6055       0.6697        0.8883        18.6372
     99       0.7679        0.6021       0.6898        0.8290        18.4940
    100       0.7858        0.5784       0.6809        0.8426        18.5352

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNetEgemaps2048(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (fc1): Sequen

###  2. Изменяем структуру модели: второй полносвязный слой делаем не 2048 нейронов, а 1792. eGeMAPS пропускаем через полносвязный слой 256 нейронов, и его конкатенируем со слоем с 1792 нейронов. <br>
Эту модель протестим без LayerNorm и с LayerNorm.


#### 2.1. Без LayerNorm

In [3]:
model = AlexNetEgemaps1792(4)
print(model)

AlexNetEgemaps1792(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
  (spec_fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=6400, out_features=1792, bias

In [4]:
filename = 'AlexNetEgemaps1792--{}_augmentation-{}.md'.format(train_ds.name, str(train_ds.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [5]:
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(valid_ds), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [6]:
net.fit(train_ds, None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.3401        1.3461       0.3807        1.3145     +  18.3485
      2       0.3788        1.3119       0.3807        1.2991     +  18.2514
      3       0.4065        1.2798       0.4938        1.1809     +  18.2304
      4       0.5176        1.1303       0.5442        1.0659     +  18.2034
      5       0.5496        1.0674       0.5543        1.0241     +  18.2525
      6       0.5605        1.0487       0.5677        1.0010     +  18.3884
      7       0.5669        1.0409       0.5610        1.0231        18.2834
      8       0.5798        1.0304       0.5857        0.9892     +  18.2765
      9       0.5728        1.0296       0.5476        1.0347        18.2134
     10       0.5747        1.0228       0.5733        1.0048        18.2231
     11       0.5890        1.0113       0.5689        0.9938        18.2409

     91       0.6719        0.8233       0.6540        0.8501     +  18.3029
     92       0.6716        0.8269       0.6484        0.8624        18.3355
     93       0.6727        0.8152       0.6495        0.8494     +  18.3730
     94       0.6719        0.8175       0.6484        0.8561        18.1030
     95       0.6781        0.8161       0.6461        0.8898        18.1880
     96       0.6688        0.8217       0.6372        0.8578        18.1495
     97       0.6809        0.8089       0.6417        0.8684        18.2003
     98       0.6697        0.8109       0.6428        0.8745        18.2144
     99       0.6716        0.8192       0.6495        0.8604        18.1024
    100       0.6761        0.7956       0.6417        0.8670        18.2522
    101       0.6795        0.8002       0.6450        0.8695        18.1520
    102       0.6828        0.7869       0.6495        0.8444     +  18.2817
    103       0.6809        0.8005       0.6461        0.8523        18.3715

    187       0.7788        0.5664       0.6473        0.8608        18.4243
    188       0.7735        0.5835       0.6607        0.8648        18.1953
    189       0.7830        0.5494       0.6484        0.8884        18.2986
Stopping since valid_loss has not improved in the last 30 epochs.


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNetEgemaps1792(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (spec_fc): Se